In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt; plt.ion()
from matplotlib import colors as mcolors
from matplotlib.image import AxesImage
from matplotlib.collections import PolyCollection, LineCollection, PathCollection
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection, Path3DCollection

from scipy.io import loadmat
from astar.astar import  AStar
from astar.environment_simple import Environment2D

def tic():
  return time.time()
def toc(tstart, nm=""):
  # print('%s took %s sec.\n' % (nm,(time.time() - tstart)))
  return (time.time() - tstart)

In [2]:
def drawMap(ax,cmap):
  '''
    Draws the matrix cmap as a grayscale image in the axes ax
    and returns a handle to the plot.
    The handle can be used later to update the map like so:
    
    f, ax = plt.subplots()
    h = drawMap(ax, cmap)
    # update cmap
    h = drawMap(h,cmap)
  '''
  if type(ax) is AxesImage:
    # update image data
    ax.set_data(cmap)
  else:
    # setup image data for the first time
    # transpose because imshow places the first dimension on the y-axis
    h = ax.imshow( cmap.T, interpolation="none", cmap='gray_r', origin='lower', \
                   extent=(-0.5,cmap.shape[0]-0.5, -0.5, cmap.shape[1]-0.5) )
    ax.axis([-0.5, cmap.shape[0]-0.5, -0.5, cmap.shape[1]-0.5])
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    return h

def drawPath2D(ax, traj):
  ''' h = drawPath2D(h,traj)
      
      traj = num_traj x num_pts x num_dim
  '''
  if(traj.ndim < 3):
    traj = traj[None,...]

  if type(ax) is LineCollection:
    ax.set_verts(traj)
  else:
    colors = [mcolors.to_rgba(c)
              for c in plt.rcParams['axes.prop_cycle'].by_key()['color']]
    h = ax.add_collection(LineCollection(traj,colors=colors))
    return h

def drawPath3D(ax, traj):
  ''' h = drawPath3D(h,traj)
      
      traj = num_traj x num_pts x num_dim
  ''' 
  if(traj.ndim < 3):
    traj = traj[None,...]

  if type(ax) is Line3DCollection:
    ax.set_verts(traj)
  else:
    colors = [mcolors.to_rgba(c)
              for c in plt.rcParams['axes.prop_cycle'].by_key()['color']]
    h = ax.add_collection(Line3DCollection(traj,colors=colors))
    return h


A* Performance on Mazes

In [3]:
testing_data_28x28 = loadmat('gridworld_28_test.mat')
total_time = 0
for map_id in range(0,1000):
    cmap = 1-testing_data_28x28["all_im_data"].reshape(1000,28,28)[map_id,:,:].T
    goal_coord = testing_data_28x28["all_states_xy"][map_id][0][-1]-1
    start_coord = testing_data_28x28["all_states_xy"][map_id][0][0]-1
    env = Environment2D(goal_coord,cmap)
    t1 = tic()
    path_cost, path, action_idx = AStar.plan(start_coord, env)
    dt = toc(t1, "A*")
    total_time+=dt

print("Average Time Taken by A*:", total_time/1000)


Average Time Taken by A*: 0.003216294527053833
